# FPSR
------------
This is an demonstration of our paper:

<p align="center" width="40%">
    <kbd><img width="40%" src="https://raw.githubusercontent.com/Joinn99/RepositoryResource/master/FPSR.svg"> </kbd>
</p>

> Tianjun Wei, Jianghong Ma, Tommy W.S. Chow. Fine-tuning Partition-aware Item Similarities for Efficient and Scalable Recommendation. In WWW 2023. [[arxiv](https://arxiv.org/abs/2207.05959)] [[Github](https://github.com/Joinn99/FPSR)]

## Preprocessing

In [1]:
#@title Environment Setting
#@markdown FPSR is implemented based on the recommendation toolbox [RecBole](https://recbole.io/).
%%capture
! pip install recbole==1.1.1

In [2]:
#@title Download Dataset
#@markdown Here we use four public datasets:<br>
#@markdown - [Amazon-cds](https://github.com/xue-pai/UltraGCN/blob/main/data/amazoncds/)
#@markdown - [Douban](https://github.com/librahu/HIN-Datasets-for-Recommendation-and-Network-Embedding/blob/master/Douban%20Book/user_book.dat)
#@markdown - [Gowalla](https://github.com/kuandeng/LightGCN/blob/master/Data/gowalla/)
#@markdown - [Yelp2018](https://github.com/kuandeng/LightGCN/blob/master/Data/yelp2018)

#@markdown We also provide processed datasets that conform to the RecBole data format, which can be downloaded [here](https://github.com/Joinn99/FPSR/tree/master/Data).

%%capture
! git clone https://github.com/Joinn99/FPSR
! cd FPSR && unzip -o "Data/*.zip" -d /content

In [3]:
#@title Configuation
#@markdown Here we list the configurations of the experiment.
#@markdown For a detailed description of the configuration, please refer to [RecBole API](https://recbole.io/docs/user_guide/config_settings.html).

CONFIG = {
    "seed": 2026,
    "reproducibility": True,

    "eval_args":{
        "split": {'RS': [0.7, 0.15, 0.15]},
        "order": 'RO',
        "group_by": 'user',
        "mode": 'full'
    },
    "topk": [10, 20],
    "metrics": ['Recall', 'NDCG'],
    "metric_decimal_place": 6,

    "data_path": 'Data',
    "load_col": {
        "inter": ['user_id', 'item_id'],
        "user": ['user_id'],
        "item": ['item_id']
    },
    "USER_ID_FIELD": 'user_id',
    "ITEM_ID_FIELD": 'item_id',

    "train_neg_sample_args":{
        "sample_num": 1
    },
    "train_batch_size": 524288,
    "eval_batch_size": 1048576,
    "filter_inter_by_user_or_item": False,
    "epochs": 1,
    "eval_step": 0,
}


### Hyperparameters
Here we list the hyperparameter setting of FPSR. To evaluate the performance of FPSR with different hyperparameters, modify the values in `FPSR_HYPER` and select `Runtime->Run after`.

In [4]:
DATASET = {
    'dataset': 'douban'
}

FPSR_HYPER = {
    "eigenvectors": 256,
    "opti_iter": 50,
    "lambda": 0.2,
    "rho": 500,
    "theta_1": 0.8,
    "theta_2": 0.1,
    "eta": 1.0,
    "tol": 5e-3,
    "tau": 0.5
}

CONFIG.update(DATASET)
CONFIG.update(FPSR_HYPER)

## Model Implementation

In [5]:
#@title FPSR
#@markdown This cell includes a complete implementation of FPSR model.
#@markdown You can find it at [Github](https://github.com/Joinn99/FPSR/blob/torch/FPSR/model.py).

r"""
FPSR
################################################
Reference:
    Tianjun Wei et al. "Fine-tuning Partition-aware Item Similarities for Efficient and Scalable Recommendation." in WWW 2023.
Reference code:
    https://github.com/Joinn99/FPSR/tree/torch
"""

import torch
import numpy as np

from recbole.utils import InputType
from recbole.utils.enum_type import ModelType
from recbole.model.abstract_recommender import GeneralRecommender

class FPSR(GeneralRecommender):
    r"""FPSR is an item-based model for collaborative filtering.

    FPSR introduces graph partitioning on the item-item adjacency graph, aiming to restrict the scale of item similarity modeling.
    Specifically, it shows that the spectral information of the original item-item adjacency graph is well in preserving global-level information.
    Then, the global-level information is added to fine-tune local item similarities with a new data augmentation strategy acted as partition-aware
    prior knowledge, jointly to cope with the information loss brought by partitioning.
    """
    input_type = InputType.POINTWISE
    type = ModelType.TRADITIONAL

    def __init__(self, config, dataset):
        super().__init__(config, dataset)

        # load parameters info
        self.eigen_dim = config['eigenvectors'] # int type: Num of eigenvectors extracted for W
        self.lambda_ = config['lambda']         # float32 type: Lambda
        self.rho = config['rho']                # float32 type: Rho
        self.theta_1 = config['theta_1']        # float32 type: Theta_1
        self.theta_2 = config['theta_2']        # float32 type: Theta_2
        self.eta = config['eta']                # float32 type: Eta
        self.tol = config['tol']                # float32 type: Threshold to filter out small values
        self.tau = config['tau']                # float32 type: Size ratio of partitions
        
        # dummy params for recbole
        self.dummy_param = torch.nn.Parameter(torch.zeros(1))   # Dummy pytorch parameters required by Recbole

        # load dataset info
        self.inter = dataset.inter_matrix(form='coo')   # User-item interaction matrix
        self.inter = torch.sparse_coo_tensor(
                            torch.LongTensor(np.array([self.inter.row, self.inter.col])),
                            torch.FloatTensor(self.inter.data),
                            size=self.inter.shape, dtype=torch.float
                        ).coalesce().to(self.device)
        
        # storage variables for item similarity matrix S
        self.S_indices = []
        self.S_values = []

        # training process
        self.update_W()   
        first_split = self.partitioning(self.V)                                                     # calaulate W and generate first split
        self.update_S(torch.arange(self.n_items, device=self.device)[torch.where(first_split)[0]])  # recursive paritioning #1
        self.update_S(torch.arange(self.n_items, device=self.device)[torch.where(~first_split)[0]]) # recursive paritioning #2
        
        self.S = torch.sparse_coo_tensor(indices=torch.cat(self.S_indices, dim=1),
                                         values=torch.cat(self.S_values, dim=0),
                                         size=(self.n_items, self.n_items)).coalesce().T.to_sparse_csr()
        del self.S_indices, self.S_values

    def _degree(self, inter_mat=None, dim=0, exp=-0.5) -> torch.Tensor:
        r"""Get the degree of users and items.
        
        Returns:
            Tensor of the node degrees.
        """
        if inter_mat is None:
            inter_mat = self.inter
        d_inv = torch.nan_to_num(torch.sparse.sum(inter_mat,dim=dim).to_dense().pow(exp), nan=0, posinf=0, neginf=0)
        return d_inv

    def _svd(self, mat, k) -> torch.Tensor:
        r"""Perform Truncated singular value decomposition (SVD) on
        the input matrix, return top-k eigenvectors.
        
        Returns:
            Tok-k eigenvectors.
        """
        _, _, V = torch.svd_lowrank(mat, q=max(4*k, 32), niter=10)
        return V[:, :k]

    def _norm_adj(self, item_list=None) -> torch.Tensor:
        r"""Get the normalized item-item adjacency matrix for a group of items.
        
        Returns:
            Sparse tensor of the normalized item-item adjacency matrix.
        """
        if item_list is None:
            vals = self.inter.values() * self.d_i[self.inter.indices()[1]].squeeze()
            return torch.sparse_coo_tensor(
                            self.inter.indices(),
                            self._degree(dim=1)[self.inter.indices()[0]] * vals,
                            size=self.inter.shape, dtype=torch.float
                        ).coalesce()
        else:
            inter = self.inter.index_select(dim=1, index=item_list).coalesce()
            vals = inter.values() * self.d_i[item_list][inter.indices()[1]].squeeze()
            return torch.sparse_coo_tensor(
                            inter.indices(),
                            self._degree(inter, dim=1)[inter.indices()[0]] * vals,
                            size=inter.shape, dtype=torch.float
            ).coalesce()
    

    def update_W(self) -> None:
        r"""Derive the global-level information metrix W, and use the eigenvector
        with the second largest eigenvalue to perform first graph partitioning.
        
        """
        self.d_i = self._degree(dim=0).reshape(-1, 1)
        self.d_i_inv = self._degree(dim=0, exp=0.5).reshape(1, -1)
        self.V = self._svd(self._norm_adj(), self.eigen_dim)

    def partitioning(self, V) -> torch.Tensor:
        r"""Perform graph bipartitioning.
        
        Returns:
            Paritioning result.
        """
        split = V[:, 1] >= 0
        if split.sum() == split.shape[0] or split.sum() == 0:
            split = V[:, 1] >= torch.median(V[:, 1])
        return split

    def update_S(self, item_list) -> None:
        r"""Derive partition-aware item similarity matrix S in each partition.
        
        """
        if item_list.shape[0] <= self.tau * self.n_items:
            # If the partition size is samller than size limit, model item similarity for this partition.
            comm_inter = self.inter.index_select(dim=1, index=item_list).to_dense()
            comm_inter = torch.mm(comm_inter.T, comm_inter)
            comm_ae = self.item_similarity(
                comm_inter,
                self.V[item_list, :],
                self.d_i[item_list, :],
                self.d_i_inv[:, item_list]
            )
            comm_ae = torch.where(comm_ae >= self.tol, comm_ae, 0).to_sparse_coo()
            self.S_indices.append(item_list[comm_ae.indices()])
            self.S_values.append(comm_ae.values())
        else:
            # If the partition size is larger than size limit, perform graph partitioning on this partition.
            split = self.partitioning(self._svd(self._norm_adj(item_list), 2))
            self.update_S(item_list[torch.where(split)[0]])
            self.update_S(item_list[torch.where(~split)[0]])
    
    def item_similarity(self, inter_mat, V, d_i, d_i_inv) -> torch.Tensor:
        r"""Update partition-aware item similarity matrix S in a specific partition.
        
        Returns:
            Partition-aware item similarity matrix of a partition.
        """
        # Initialize
        Q_hat = inter_mat + self.theta_2 * torch.diag(torch.pow(d_i_inv.squeeze(), 2)) + self.eta
        Q_inv = torch.inverse(Q_hat + self.rho * torch.eye(inter_mat.shape[0], device=self.device))
        Z_aux = (Q_inv @ Q_hat @ (torch.eye(inter_mat.shape[0], device=self.device) - self.lambda_ * d_i * V @ V.T * d_i_inv))
        del Q_hat
        Phi = torch.zeros_like(Q_inv, device=self.device)
        S = torch.zeros_like(Q_inv, device=self.device)
        for _ in range(50):
            # Iteration
            Z_tilde = Z_aux + Q_inv @ (self.rho * (S - Phi))
            gamma = torch.diag(Z_tilde) / (torch.diag(Q_inv) + 1e-10)
            Z = Z_tilde - Q_inv * gamma                                 # Update Z
            S = torch.clip(Z + Phi - self.theta_1 / self.rho, min=0)    # Update S 
            Phi += Z - S                                                # Update Phi
        return S

    def forward(self):
        pass

    def calculate_loss(self, interaction):
        return torch.nn.Parameter(torch.zeros(1))

    def predict(self, interaction):
        user = self.inter.index_select(dim=0, index=interaction[self.USER_ID]).to_dense()
        item = self.S.index_select(dim=1, index=interaction[self.ITEM_ID]).to_dense()
        d_i_inv = self.d_i_inv[:, interaction[self.ITEM_ID]]
        V = self.V[interaction[self.ITEM_ID], :]
        
        r = torch.mul(item.T, user).sum(dim=-1)
        r += self.lambda_ * torch.mul(user * self.d_i.T @ self.V, V * d_i_inv.T).sum(dim=-1)
        return r

    def full_sort_predict(self, interaction) -> torch.Tensor:
        user = self.inter.index_select(dim=0, index=interaction[self.USER_ID]).to_dense()
        r = torch.sparse.mm(self.S, user.T).T
        r += self.lambda_ * user * self.d_i.T @ self.V @ self.V.T * self.d_i_inv
        return r

## Running

In [6]:
r"""
Code reference: https://recbole.io/docs/developer_guide/customize_models.html
"""

import warnings
from recbole.utils import init_seed
from recbole.trainer import Trainer
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

warnings.filterwarnings('ignore')

print('='* 10 + 'Preprocessing' + '=' * 10)
config = Config(model=FPSR, config_dict=CONFIG)
init_seed(config['seed'], config['reproducibility'])
print("Configuration loading complete.")

# dataset filtering
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
print("Data loading complete.")

print('='* 12 + 'Training' + '=' * 13)
print("(This step may last several minutes)")
# model loading and initialization
model = FPSR(config, train_data.dataset).to(config['device'])

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
_, _ = trainer.fit(train_data, valid_data)

print("Training complete.")

print('='* 11 + 'Evaluation' + '=' * 12)
# model evaluation
test_result = trainer.evaluate(test_data)
print("Evaluation results:")
for metric, value in test_result.items():
  print('{:10s}: {:.6f}'.format(metric.upper(), value))

==========Preprocessing==========
Configuration loading complete.
Data loading complete.
============Training=============
(This step may last several minutes)
Training complete.
===========Evaluation============
Evaluation results:
RECALL@10 : 0.157929
RECALL@20 : 0.208824
NDCG@10   : 0.192254
NDCG@20   : 0.194117
